In [12]:
import numpy as np
import pandas as pd
import os
import aniposelib
import scipy.io

In [224]:
data_path = '/mnt/soma_cifs/Iyer/DANNCE_Files/tadpole_stage57_2022-11-04/DLTdv8_data_xyzpts.csv';
calib_path = '/mnt/soma_cifs/Iyer/Filming/02112022a/calibrationwater_1515/calibration.toml';
label3d_path = '/mnt/soma_cifs/Iyer/DANNCE_Files/tadpole-stage57_Nov2022a/label3d_data.mat';
n_frames = 88
n_pts = 13

In [225]:
def add_pts(data_path, calib_path, label3d_path, n_frames, n_pts):
    df = pd.read_csv(data_path);
    calibration_ = aniposelib.cameras.CameraGroup.load(calib_path);
    matfile_ = scipy.io.loadmat(label3d_path)['label3d_data'];

    df_np = df.to_numpy().reshape(-1, 3)
    df_cam = calibration_.project(df_np)
    df_cam = np.rint(df_cam).astype('int32')

    for cam_num in [0,1,2]:
        matfile_['labelData'][0][0][cam_num][0]['data_2d'][0][0] = df_cam[cam_num].reshape(n_frames, n_pts*2)
        matfile_['labelData'][0][0][cam_num][0]['data_3d'][0][0] = df.to_numpy()
    scipy.io.savemat(label3d_path, {'label3d_data':matfile_})

In [226]:
add_pts(data_path, calib_path, label3d_path, n_frames, n_pts)